In [70]:
import json
from landsatxplore.api import API
import getpass
import pandas as pd
import geopandas as gpd
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from hublib.ui import FileUpload
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import FileUpload, Checkbox, FloatSlider, Dropdown, Button, Output, VBox, DatePicker
import urllib.request
from io import BytesIO
from zipfile import ZipFile
from datetime import datetime
import os
from IPython.display import display, HTML

In [71]:
import hublib.ui as ui
from hublib.ui import HideCodeButton, RunAllButton

In [72]:
RunAllButton()

Button(description='Run All Cells', style=ButtonStyle())

In [73]:
RunAllButton(label="Run Cells", icon='fa-play-circle-o', style='danger', tooltip='Run All Cells', hide=True)

Button(button_style='danger', description='Run Cells', icon='play-circle-o', style=ButtonStyle(), tooltip='Run…

In [74]:
HideCodeButton()

Button(description='Hide Code Cells', style=ButtonStyle())

In [76]:
# username_input = widgets.Text(description='Enter your Earth Explorer username: ')
# password_input = widgets.Password(description='Enter your Earth Explorer password: ')
username = input('Enter your Earth Explorer username: ')
password = getpass.getpass('Enter your Earth Explorer password: ')
# bbox_file_input = widgets.FileUpload(description='Upload Bbox Shapefile')

Enter your Earth Explorer username: maggiechen259
Enter your Earth Explorer password: ········


In [77]:
allowed_extensions = ['.shp', '.gpkg']

file_upload = FileUpload(
    description="Upload Shapefile",
    accept=','.join(allowed_extensions),
    multiple=False
)


In [78]:
# output_dir = './box'
# local_file_path = os.path.join(output_dir, f'{ID}.shp')


In [79]:
# # Attach the handle_file_upload function to the upload button's on_click event
# upload_button.on_click(handle_file_upload)

In [80]:
message = "Please choose within two weeks"
display(HTML(f"<p style='color:red'>{message}</p>"))

In [81]:
start_date = DatePicker(
    description='Start Date',
    disabled = False,
    layout={'width': '300px'}
)
# start_date

In [82]:
end_date = DatePicker(
    description='End Date',
    disabled = False,
    layout={'width': '300px'}
)
# end_date

In [83]:
# Create multiple checkboxes
check1 = ui.Checkbox('Landsat 5', desc='Landsat 5', value=False)
check2 = ui.Checkbox('Landsat 7', desc='Landsat 7', value=False)
check3 = ui.Checkbox('Landsat 8', desc='Landsat 8', value=False)
check4 = ui.Checkbox('Landsat 9', desc='Landsat 9', value=False)

In [85]:

# Display the checkboxes in a horizontal layout
landsat = widgets.VBox([check1, check2, check3,check4])

# Cloud cover
cloud_cover = ui.Number(
        name='Max Cloud Cover',
        description="Cloud Cover",
        units='%',
        min=0,
        max=500,
        value=0
)

composite_color = ui.Dropdown(
            name='Composite Color',
            description="Choose the composite color",
            value='True Color',
            options={'True Color', 'False Color'},
#             cb=plot
        )

In [87]:
# Output widget to display the result
result_output = Output()

In [88]:
# # Button to trigger the search and display results
# search_button = Button(description='Display')

In [89]:
# Function to handle button click event
def on_search_button_click(b):
    # Retrieve the file path from the FileUpload widget
    uploaded_files = file_upload.value
    if uploaded_files:
        shapefile_path = list(uploaded_files.keys())[0]
        
        # Read the uploaded shapefile
        bbox = gpd.read_file(shapefile_path)
        bounds = bbox.bounds.iloc[0].to_list()
        
        # Retrieve other widget values
        landsat_options = [check1.value, check2.value, check3.value, check4.value]
        cloud_cover_value = cloud_cover.value
        composite_color_value = composite_color.value
        
        # Perform further processing based on the retrieved values
        with result_output:
            print(f"Shapefile Path: {shapefile_path}")
            print(f"Landsat Options: {landsat_options}")
            print(f"Cloud Cover: {cloud_cover_value}%")
            print(f"Composite Color: {composite_color_value}")

In [90]:
# Button to trigger the search and display results
search_button = Button(description='Display')

In [91]:
# #Attach the function to the button click event
# display_button.on_click(on_search_button_click)

In [92]:
# Display the widgets
VBox([file_upload,
#       upload_button, 
#       output,
      start_date,
      end_date,
      VBox([check1, check2, check3, check4]),
      cloud_cover, 
      composite_color, 
      search_button,
      result_output])


In [97]:
def handle_file_upload(change):
    with output:
        # Clear previous output
        output.clear_output()

        # Check if any files are uploaded
        if file_upload.value:
            # Get the first uploaded file content
            uploaded_file_content = next(iter(file_upload.value.values()))['content']

            # Create a file-like object from the uploaded content
            file_like_object = BytesIO(uploaded_file_content)

            # Extract the zip file if it's a shapefile archive
            with ZipFile(file_like_object, 'r') as zip_ref:
                # Assuming the shapefile is in the root of the zip file
                shapefile_name = [name for name in zip_ref.namelist() if name.endswith('.shp')][0]

                # Read the shapefile using geopandas
                bbox = gpd.read_file(BytesIO(zip_ref.read(shapefile_name)))

            # Extract the bounds from the shapefile
            bounds = bbox.bounds.iloc[0].to_list()

            print("File uploaded successfully.")
            print("Bounds:", bounds)

In [98]:
# Process uploaded bbox shapefile
# bbox_filename = input('Enter the name of the uploaded bounding box shapefile (with extension): ')

uploaded_filename = next(iter(file_upload.value))
print(f"Uploaded file name: {uploaded_filename}")

bbox_file_path = f'bbox/{uploaded_filename}'



Uploaded file name: helheim.gpkg


In [105]:
#upload area of interest
upload_directory = './bbox'
os.makedirs(upload_directory, exist_ok=True)

try:
        if file_upload.value:
        # Iterate over uploaded files
            for filename, file_info in file_upload.value.items():
                # Save the file to the specified directory
                file_path = os.path.join(upload_directory, filename)
                with open(file_path, 'wb') as f:
                    f.write(file_info['content'])

                print(f"File '{filename}' saved to '{upload_directory}'")
            
except Exception as e:
        print(f'Uploaded failed. Error: {str(e)}')


# bbox = gpd.read_file(file_upload.value[0])
bbox = gpd.read_file(bbox_file_path)
bbox = bbox.to_crs('EPSG:4326')
bounds = bbox.bounds.iloc[0].to_list()

File 'helheim.gpkg' saved to './bbox'


In [106]:
def display_table():
    df_scenes = pd.DataFrame(scenes)
    df_scenes = df_scenes[['display_id','wrs_path', 'wrs_row','satellite','cloud_cover','acquisition_date']]
    df_scenes.sort_values('acquisition_date', ascending=False, inplace=True)
    df_scenes['acquisition_date'] = pd.to_datetime(df_scenes['acquisition_date'])
    display(df_scenes)

In [108]:
#Landsat 5 TM Collection 2 Level 1	landsat_tm_c2_l1
# Landsat 5 TM Collection 2 Level 2	landsat_tm_c2_l2
# Landsat 7 ETM+ Collection 2 Level 1	landsat_etm_c2_l1
# Landsat 7 ETM+ Collection 2 Level 2	landsat_etm_c2_l2
# Landsat 8 Collection 2 Level 1	landsat_ot_c2_l1
# Landsat 8 Collection 2 Level 2	landsat_ot_c2_l2
# Landsat 9 Collection 2 Level 1	landsat_ot_c2_l1
# Landsat 9 Collection 2 Level 2	landsat_ot_c2_l2

# dic = {5:['landsat_tm_c2_l1','landsat_tm_c2_l2'],
#        7:['landsat_etm_c2_l1','landsat_etm_c2_l2'],
#        8:['landsat_ot_c2_l1','landsat_ot_c2_l2'],
#        9:['landsat_ot_c2_l1','landsat_ot_c2_l2']}

dic = {5:'landsat_tm_c2_l1',
       7:'landsat_etm_c2_l1',
       8:'landsat_ot_c2_l1',
       9:'landsat_ot_c2_l1'}
landsats = [5,7,8,9]


# #Initialize a new API instance

# Get the values of checkboxes within the VBox
checkbox_values = [child.value for child in landsat.children]

# print(checkbox_values)
for index, value in enumerate(checkbox_values):
    if value == True:
        landsat_id = landsats[index]
print(landsat_id)
print(dic[landsat_id])

# if landsat.value in dic:
#     landsat_dataset_id = dic[landsat.value]
    
api = API(username, password)

# Perform a request
response = api.request(endpoint="dataset-catalogs")
print(response)

# Search for Landsat TM scenes
print(start_date.value)
print(end_date.value)
scenes = api.search(
#     dataset='landsat_ot_c2_l1',
    dataset = str(dic[landsat_id]),
#     dataset=str(checkbox_values[0]),
    bbox = bounds,
    start_date = str(start_date.value),
    end_date = str(end_date.value),
    max_cloud_cover= str(cloud_cover.value),
#     composite_color = composite_color.value,
#     landsat = [checkbox.value for checkbox in landsat.children]
)
print(f"{len(scenes)} scenes found.")

# Process the result
for scene in scenes:
    print(scene['acquisition_date'].strftime('%Y-%m-%d'))
    # Write scene footprints to disk
    fname = f"{scene['landsat_product_id']}.geojson"
    with open(fname, "w") as f:
        json.dump(scene['spatial_coverage'].__geo_interface__, f)

# log out
api.logout()



8
landsat_ot_c2_l1
{'EE': 'EarthExplorer', 'GV': 'GloVis', 'HDDS': 'HDDS Explorer'}
2022-04-20
2022-06-20
31 scenes found.
2022-06-19
2022-06-17
2022-06-17
2022-06-11
2022-06-10
2022-06-10
2022-06-10
2022-06-10
2022-06-09
2022-06-09
2022-06-08
2022-06-08
2022-06-07
2022-06-04
2022-06-03
2022-06-02
2022-05-30
2022-05-14
2022-05-09
2022-05-08
2022-05-08
2022-05-07
2022-05-07
2022-05-06
2022-05-01
2022-04-30
2022-04-30
2022-04-29
2022-04-23
2022-04-23
2022-04-20


In [110]:
# Create a DataFrame from the scenes
df_scenes = pd.DataFrame(scenes)
df_scenes = df_scenes[['display_id','wrs_path', 'wrs_row','satellite','cloud_cover','acquisition_date']]
df_scenes.sort_values('acquisition_date', ascending=True, inplace=True)
df_scenes['acquisition_date'] = pd.to_datetime(df_scenes['acquisition_date'])

dates = []
for scene in scenes:
    dates.append(scene['acquisition_date'].strftime('%Y-%m-%d'))
#     print(scene['acquisition_date'].strftime('%Y-%m-%d'))
df_scenes




,display_id,wrs_path,wrs_row,satellite,cloud_cover,acquisition_date
30,LC09_L1GT_001013_20220420_20230421_02_T2,1,13,9,1,2022-04-20
28,LC08_L1TP_231013_20220423_20220428_02_T1,231,13,8,2,2022-04-23
29,LC08_L1TP_231014_20220423_20220428_02_T1,231,14,8,17,2022-04-23
27,LC09_L1TP_233013_20220429_20230418_02_T1,233,13,9,9,2022-04-29
26,LC08_L1TP_232014_20220430_20220504_02_T1,232,14,8,16,2022-04-30
25,LC08_L1TP_232013_20220430_20220504_02_T1,232,13,8,14,2022-04-30
24,LC09_L1TP_231013_20220501_20230418_02_T1,231,13,9,2,2022-05-01
23,LC09_L1GT_001013_20220506_20230417_02_T2,1,13,9,0,2022-05-06
22,LC08_L1TP_233014_20220507_20220512_02_T1,233,14,8,0,2022-05-07
21,LC08_L1TP_233013_20220507_20220512_02_T1,233,13,8,0,2022-05-07


In [111]:
choose_date = [ui.Checkbox(date, value=False, description=date) for date in dates]

display_dates = widgets.VBox(choose_date)


# # Display the checkboxes in a horizontal layout
# landsat = widgets.VBox([check1, check2, check3,check4])

display(display_dates)

In [112]:
checkbox_dates = [child.value for child in display_dates.children]
# print(checkbox_dates)
# print(dates)

selected_dates_index = []

for index, date in enumerate(checkbox_dates):
    if date == True:
        chosen_date=dates[index]
        chosen_date_index = index
        selected_dates_index.append(index)
        print(chosen_date)
        print(chosen_date_index)

print(selected_dates_index)


2022-06-10
5
[5]


In [113]:
# # Attach the function to the button click event
# display_button.on_click(display_table)
# # Attach the function to the button click event using a lambda function
# # display_button.on_click(lambda _: display_table())


In [114]:
# exe_button = widgets.Button(
#     description = 'Run Analysis',
#     disabled = False,
#     button_style = 'success'
# )
# exe_button.on_click(execute)

In [115]:
from landsatxplore.earthexplorer import EarthExplorer
import os
import time
import requests

for index in selected_dates_index:
# Initialize the API
    api = EarthExplorer(username, password)

    # Select the first scene
    # Assuming the first scene ID is stored in the variable first_scene_id
    ID = df_scenes['display_id'].iloc[index]


    # Get the Earth Explorer website URL for the scene
    api_scene_url = f'https://earthexplorer.usgs.gov/order/process?dataset_name=LANDSAT_8_C1&scene_id={ID}&node=INVSVC'


    # Open the scene URL in a web browser for manual download
    # print(f'You can manually download the scene from the following URL:')
    # print(api_scene_url)

    # Define the local file path
    output_dir = './data'
    local_file_path = os.path.join(output_dir, f'{ID}.tar')



    # Download the scene using requests
    try:
        response = requests.get(api_scene_url)
        if response.status_code == 200:
            with open(local_file_path, 'wb') as f:
                f.write(response.content)
            print(f'{ID} successfully downloaded.')
        else:
            print(f'{ID} download failed. HTTP status code: {response.status_code}')
    except Exception as e:
        print(f'{ID} download failed. Error: {str(e)}')

# api.logout()


LC08_L1TP_232013_20220430_20220504_02_T1 successfully downloaded.


In [ ]:
import tarfile

ID = 'LC08_L2SP_010010_20221004_20221012_02_T1'
file_path = './data/{}.tar'.format(ID)

try:
    tar = tarfile.open(file_path)
    tar.extractall('./data/{}'.format(ID))
    tar.close()
    print('Extraction successful')
except Exception as e:
    print(f'Error: {e}')
    
#unzip twice 


In [ ]:
# import tarfile

# # Extract files from tar archive
# tar = tarfile.open('./data/{}.tar'.format(ID))
# tar.extractall('./data/{}'.format(ID))
# tar.close()

In [75]:
# # called when all files finish uploading
# def done_cb(w, name):
#     # Do something with the files here...
#     # We just print their names
#     print("%s uploaded" % name)
    
#     # reset clears and re-enables the widget for more uploads
#     # You may want to do this somewhere else, depending on your GUI
#     w.reset()